# Information

- Full paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4374466/

# Load & config libraries

In [ ]:
import mne
import scipy.io as sio
import numpy    as np

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

mne.set_log_level('INFO')

# Load file

In [ ]:
# Load matlab file
# https://docs.scipy.org/doc/scipy/reference/tutorial/io.html
# Load the MATLAB structs as python objects rather than numpy structured arrays (struct_as_record=False)
mat = sio.loadmat('/Users/juanm/Downloads/Proyecto Final/Continous ErrP Dataset/Subjects/S01.mat', struct_as_record=False, squeeze_me=True, chars_as_strings=True)

print('Loaded subject {} file'.format(mat['info'].subject))

# Load signals

In [ ]:
ch_names = mat['info'].channelnames.tolist()
ch_types = ['eeg'] * 28 + ['eog'] * 3 # 28 EEG & 3 EOG channels
sampling_rate  = mat['info'].samplingrate_Hz

# https://martinos.org/mne/stable/generated/mne.create_info.html
sig_info = mne.create_info(ch_names, sampling_rate, ch_types)

# Create MNE objects from data array
# https://martinos.org/mne/stable/auto_examples/io/plot_objects_from_arrays.html
sig_data = mat['eeg_signal']
sig_mne  = mne.io.array.RawArray(sig_data.T, sig_info)

# Set missing montage (electrode positions)
# https://martinos.org/mne/stable/generated/mne.io.Raw.html#mne.io.Raw.set_montage
# https://mne-tools.github.io/stable/generated/mne.channels.Montage.html
# Electrodes are named and positioned according to the international 10-20 system (94+3 locations)
sig_mne.set_montage('standard_1020')

print('Loaded subject {} data'.format(mat['info'].subject))

# Plot signals

In [ ]:
# Plot sensors
sig_mne.plot_sensors()

In [ ]:
# Only include EEG channels
picks = mne.pick_types(sig_info, eeg=True)

In [ ]:
# Compute the power spectral density of raw data
# https://martinos.org/mne/stable/auto_examples/time_frequency/plot_compute_raw_data_spectrum.html
sig_mne.plot_psd(picks=picks)

In [ ]:
# Low-pass filter data
sig_mne.filter(l_freq=1.0, h_freq=20.0)

# Plot the power spectral density across filtered channels
sig_mne.plot_psd(picks=picks)

In [ ]:
# Plot raw data of first 10 channels
sig_mne.plot(n_channels=10, scalings='auto', block=True)

In [ ]:
# start, stop = raw.time_as_index([100, 115])  # 100 s to 115 s data segment
# data, times = raw[:, start:stop]
# print(data.shape)
# print(times.shape)
# data, times = raw[2:20:3, start:stop]  # access underlying data
# raw.plot()

# Analize error

## Load error

In [ ]:
# Add ee event channel
ch_names_wee = ch_names + ['t_type']
ch_types_wee = ch_types + ['misc']
sig_wee_info = mne.create_info(ch_names_wee, sampling_rate, ch_types_wee)

# Create mne's info/raw with ee event channel
ee_data      = mat['marker'].execution_error
sig_wee_data = np.c_[sig_data, ee_data] # https://stackoverflow.com/q/20978757/3349531
sig_wee_mne  = mne.io.array.RawArray(sig_wee_data.T, sig_wee_info)
sig_wee_mne.set_montage('standard_1020')

# Find events & epochs
ee_events      = mne.find_events(sig_wee_mne, stim_channel='t_type')
sig_wee_epochs = mne.Epochs(sig_mne, ee_events, tmin=0, tmax=2)
print('Loaded (#Epochs, #Channels, #Times) = {}'.format(sig_wee_epochs.get_data().shape))

In [ ]:
# Plot evoked/avg
sig_wee_epochs_evoked = sig_wee_epochs.average()
sig_wee_epochs_evoked.plot(spatial_colors=True, time_unit='s', gfp=True)

In [ ]:
# TODO: Consultar que hace esto

# Find events & epochs
alt_sig_wee_epochs = mne.Epochs(sig_mne, ee_events, {'first': 1}, tmin=0, tmax=2)
print('Loaded (#Epochs, #Channels, #Times) = {}'.format(alt_sig_wee_epochs.get_data().shape))

# Plot evoked/avg
alt_sig_wee_epochs_evoked = alt_sig_wee_epochs.average()
alt_sig_wee_epochs_evoked.plot(spatial_colors=True, time_unit='s', gfp=True)